### Twitter Scrapper 

In [ ]:

import time
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import pandas as pd
        



def web_scrapping(links_list,filesname_list):
    try:
        time.sleep(1)  # Allow 2 seconds for the web page to open
        scroll_pause_time = 5  # You can set your own pause time. My laptop is a bit slow so I use 1 sec
        #driver = webdriver.Chrome(ChromeDriverManager().install())
       # screen_height = driver.execute_script("return window.screen.height;")  # get the screen height of the web
        
        check = 0
        

        tweets_list = []   # list of products
        for j,link in enumerate(links_list):
            driver = webdriver.Chrome(ChromeDriverManager().install())
            screen_height = driver.execute_script("return window.screen.height;")  # get the screen height of the web
            #print("screen_height>>>>>>>>> ",screen_height)
            driver.get(link)
            file_name = filesname_list[j]
            flag = False
            print("Starting File >> ",filesname_list[j])
            time.sleep(15)
            i = 1
            while True:

                # scroll one screen height each time
                driver.execute_script("window.scrollTo(0, {screen_height}*{i});".format(screen_height=screen_height, i=i))
                i += 1
                time.sleep(scroll_pause_time)
                # update scroll height each time after scrolled, as the scroll height can change after we scrolled the page
                scroll_height = driver.execute_script("return document.body.scrollHeight;")
                #print("scroll_height >>>> ",scroll_height)
                # Break the loop when the height we need to scroll to is larger than the total scroll height

                soup = BeautifulSoup(driver.page_source, "html.parser")



                for parent in soup.find_all("div", class_="css-901oao r-18jsvk2 r-37j5jr r-a023e6 r-16dba41 r-rjixqe r-bcqeeo r-bnwqim r-qvutc0"):
                    print(i,">>>",parent.text)
                    tweets_list.append(parent.text)

                tweets_list = list(set(tweets_list))
                check = check + 1
                if flag == True:
                    df1 =pd.read_csv(file_name)
                    df1.drop_duplicates(inplace = True,keep="first")
                else:
                    df1 = pd.DataFrame()
                    flag  = True

                df2 = pd.DataFrame({"new_colum":tweets_list})
                df2.drop_duplicates(inplace = True)

                frames = [df1, df2]

                result = pd.concat(frames,axis=0)    
                #print("\n Before >>",len(result))
                result.dropna(how='all',inplace=True)
                print("After >>",len(result))
                result.to_csv(file_name, index=False)
                tweets_list = []

                if (screen_height) * i > scroll_height:
                    print("Ending File >> ",filesname_list[j])
                    break
    except Exception as e:
        print("[!] ",e)


In [ ]:
#calling the scrapper to gathe the data for cricket tweets
links_list = ["https://twitter.com/search?q=%23ipl&src=typed_query&f=top","https://twitter.com/search?q=%23PSL2022&src=typeahead_click&f=top","https://twitter.com/search?q=%23cricket&src=typed_query&f=top","https://twitter.com/search?q=%23movies&src=typed_query&f=top","https://twitter.com/search?q=%23netflix&src=typed_query&f=top","https://twitter.com/search?q=%23hollywood%23movies&src=typed_query&f=top"]
files_name = ["ipl_latest1.csv","psl_latest1.csv","cricket_latest1.csv","movies_latest1.csv","netflix_latest1.csv","hollywoodMovies1.csv"]
files_name = files_name[::-1]
links_list = links_list[::-1]
web_scrapping(links_list,files_name)